In [1]:
from mip import *

In [2]:
file = open("./Instances/inst03.dat", "r")
lines = file.readlines()
file.close()
m = int(lines[0].rstrip(''))
n = int(lines[1].rstrip(''))
load = [int(x) for x in lines[2].split(' ')]
size = [int(x) for x in lines[3].split(' ')]
instance = [[int(x) for x in line.rstrip().split(' ') ]for line in lines[4:]]
o = n
print(m)
print(n)
print(load)
print(size)
for i in range(n+1):
    for j in range(n+1):
        print(instance[i][j], end = " ")
    print()

3
7
[15, 10, 7]
[3, 2, 6, 8, 5, 4, 4]
0 3 3 6 5 6 6 2 
3 0 6 3 4 7 7 3 
3 4 0 7 6 3 5 3 
6 3 7 0 5 6 7 4 
5 4 6 3 0 3 3 3 
6 7 3 6 3 0 2 4 
6 7 5 6 3 2 0 4 
2 3 3 4 3 4 4 0 


In [3]:
model = Model(sense=MINIMIZE, solver_name=CBC)

In [ ]:
# 2 variable model

# assignment[i,j] = 1 ---> item i assigned to courier j
assignment = [[model.add_var("assignment[%d,%d]" % (i, j), var_type=BINARY) for j in range(m)] for i in range(n)]

# tour[i][j] = y ---> courier i at j-th route has item y assigned
tour = [[model.add_var("tour_assignment[%d,%d]" % (i, j), var_type=INTEGER, lb=0, ub=n) for j in range(n+2)] for i in range(m)]

In [5]:
# 3 variable model
# assignment[i,j] = 1 ---> item i assigned to courier j
assignment = [[model.add_var("assignment[%d,%d]" % (i, j), var_type=BINARY) for j in range(m)] for i in range(n)]

# tour[i][j] = x ---> courier i dispatch at j-th route takes x
tour = [[model.add_var("tour[%d,%d]" % (i, j), var_type=INTEGER) for j in range(n+2)] for i in range(m)]

# tour_assignment[i][j] = y ---> courier i at j-th route has item y assigned
tour_assignment = [[model.add_var("tour_assignment[%d,%d]" % (i, j), var_type=INTEGER, lb=0, ub=n) for j in range(n+2)] for i in range(m)]

# tour[c,i,j] == 1 ---> courier c performed movement from i to j
# tour = model.add_var_tensor((m, n+1, n+1), "tour", var_type=BINARY)

# Objective function
maxDist = model.add_var("maxDist", var_type=INTEGER)                                                                              
model.objective = maxDist

In [ ]:
# Matrix model
# tour[c,i,j] == 1 ---> courier c performed movement from i to j
tour = model.add_var_tensor((m, n+1, n+1), "tour", var_type=BINARY)

In [15]:
# Constraints for matrix model

# Each item is assigned to exactly one courier
for i in range(n):
    model += xsum([tour[c][i][j] for c in range(m) for j in range(n)]) == 1

# Each courier cannot load more than his capacity
for c in range(m):
    model += xsum([tour[c][i][j]*size[j] for i in range(n+1) for j in range(n)]) <= load[c]

# Initial and final destination are the same
for c in range(m):
    model += xsum([tour[c][n][j] for j in range(n)]) == 1
    model += xsum([tour[c][j][n] for j in range(n)]) == 1


# distList = [xsum([tour[c][j] for j in range(n+1)]) for c in range(m)]                                                         
distList = [xsum(instance[i][j]*tour[c][i][j] for i in range(n+1) for j in range(n+1)) for c in range(m)]                                                         
for c in range(m):
    model += maxDist >= distList[c]

In [6]:
# Constraints for 3 variable model

# Each item is assigned to exactly one courier
for i in range(n):
    model += xsum([assignment[i][j] for j in range(m)]) == 1

# Each courier cannot load more than his capacity
for c in range(m):
    model += xsum([assignment[i][c]*size[i] for i in range(n)]) <= load[c]

# Initial and final destination are the same
for c in range(m):
    model += tour_assignment[c][0] == o
    model += tour_assignment[c][n+1] == o

# Assigned items must be in courier's tour, Link tour_assignment to assignment, if assignment[i][j] == 1 then tour_assignment[i][k] == j for all k
for c in range(m):
    for i in range(n):
        if assignment[i][c]:
            model += xsum([tour_assignment[c][j] == i for j in range(n)]) == 1

# Link tour to instance values

# Objective function constraint
distList = [xsum([tour[c][j] for j in range(n+1)]) for c in range(m)]                                                         
for c in range(m):
    model += maxDist >= distList[c]

In [33]:
status = model.optimize(max_seconds=300)

if status == OptimizationStatus.OPTIMAL:
    for c in range(m):
        print(f"Courier {c}")
        out = []
        for i in range(n+1):
            print([tour[c][i][j].x for j in range(n+1)])
            for j in range(n+1):
                if tour[c][i][j].x :
                    out.append(j)
        # print(out)
            

Clp0000I Optimal - objective value 6

Starting MIP optimization
Cgl0003I 7 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 19 rows, 151 columns (151 integer (150 of which binary)) and 425 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 4.146%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0045I MIPStart provided solution with cost 7
Cbc0012I Integer solution of 7 found by Reduced search after 0 iterations and 0 nodes (0.00 seconds)
Cbc0045I Nauty did not find any useful orbits in time 0.000177
Cbc0006I The LP relaxation is infeasible or too expensive
Cbc0013I At root node, 0 cuts changed objective from 6 to 6 in 1 passes
Cbc0014I Cut generator 0 (Probing) - 1 row cuts average 0.0 elements, 6 column cuts (6 active)  in 0.000 seconds - new frequency is 1
Cbc0014I Cut generator 1 (Gomory) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.000 seconds - new frequency is -100
Cbc0014I Cut g